In [1]:
# coding: utf-8
import urllib
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import os
import re
import csv
import sys
import time
import datetime
import threading

# 设置请求头
head = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'
}


In [2]:
futures_code = []

def decode(url):
    req = urllib.request.Request(url,headers=head)
    r = urllib.request.urlopen(req).read()
    soup = BeautifulSoup(r.decode('utf-8'))
    return soup

def re_findall(context, list_):
    a = re.findall(context,html.body.find_all("script")[5].get_text().strip('\r').strip('\n'))
    for i in a:
        list_.append(i.split(":")[1])
        
def parse(html):
    #主帖相关
    if html.body.find_all("div", class_="zwcontentmain xeditor") == []: pass
    else: 
        reviews.append(html.body.find_all("div", class_="zwcontentmain xeditor")[0].get_text()) #主帖内容
        check_time.append(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))) #查询时间
        for i in html.body.find_all('div',{'id':'zwcontt'}): #主贴发帖人信息
            for item in i.find_all('div',class_= 'data'):
                user_id.append(json.loads(item.attrs['data-json'])["user_id"])
                user_nickname.append(json.loads(item.attrs['data-json'])["user_nickname"])
                user_age.append(json.loads(item.attrs['data-json'])["user_age"])
                influence_power.append(json.loads(item.attrs['data-json'])["user_influ_level"])
        user_link = []
        for i in html.body.find_all('div',{"id":"zwconttb"}): #主帖发帖人链接
            for item in i.find_all('strong'):
                user_link.append(item.find('a').attrs['href'])
        for i in html.body.find_all('script')[2]: #主帖所在股吧
            stockbar_name=re.findall('stockname = "\w+"',i)[0].split()[-1]
        #有无图片
        if re.findall("img", str(html.body.find_all("div", class_ = "zwcontentmain xeditor"))) != []:
            has_picture.append(1)
        else: has_picture.append(0)
        #发帖时间
        for i in html.body.find_all('div',{"id":"zwcontt"}):
            for item in i.find_all('div',class_="zwfbtime"):
                publish_time.append(re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}',item.get_text())[0])
        for i in html.body.find_all('div',{"id":"zwlist"}):
            for item in i.find_all('div',class_="zwlitime"):
                publish_time.append(re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}',item.get_text())[0])
        #主帖阅读、点赞、评论量
        re_findall('"post_like_count":\d+',like_count)
        re_findall('"post_click_count":\d+',post_click_count)
        if html.body.find_all('div', {"id":"zwcontab"}) != []:
            for item in html.body.find_all('div', {"id":"zwcontab"}):
                a = item.find_all('span', class_="comment_num")[0].get_text()
                post_comment_count.append(int(re.findall(r'\d+',a)[0]))
        else: post_comment_count.append(0)
        #热评数量，用于去除热评与全评中的重复项
        num_hot_comments = 0
        for item in html.body.find_all('div',{"id":"zwcontab_hot"}):
            a = item.find_all('span',class_='comment_num')[0].get_text()
            num_hot_comments = int(re.findall(r'\d+',a)[0])
        reply_is_author.append("true") #是否为主帖

        #评论区内容
        #因为热门评论与全部评论重复，所以只选取全部评论爬取
        for i in html.body.find_all('div',{"id":"zwlist"}):
            if i.get_text()=='\n':pass
            else:
                for item in i.find_all('span',class_="zwnick"): #子帖发帖人链接
                    user_link.append(item.find('a').attrs['href'])
                for item in (i.find_all('div',{"class":"zwlitext stockcodec"})): #评论内容和图片
                    if not item.find("img"):
                        has_picture.append(0)
                        if item.get_text().strip('\n') != '': reviews.append(item.get_text().strip('\n'))
                        else: reviews.append('nan')
                    else:
                        has_picture.append(1)
                        if item.get_text().strip('\n') != '': reviews.append(item.get_text().strip('\n'))
                        else: reviews.append('nan')

                for item in i.find_all('div',class_= 'data'):
                    user_id.append(json.loads(item.attrs['data-json'])["user_id"])
                    user_nickname.append(json.loads(item.attrs['data-json'])["user_nickname"])
                    user_age.append(json.loads(item.attrs['data-json'])["user_age"])
                    influence_power.append(json.loads(item.attrs['data-json'])["user_influ_level"])

                    check_time.append(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))) #查询时间
                    #子帖阅读及评论量置nan
                    post_comment_count.append('nan')
                    post_click_count.append('nan')
            #子帖点赞量    
            tmp1 = re.findall('"reply_like_count":\d+',html.body.find_all('script')[5].get_text())[num_hot_comments:]
            for i in tmp1:
                like_count.append(i.split(":")[1])
            tmp2 = re.findall('"reply_is_author":\w+',html.body.find_all('script')[5].get_text())[num_hot_comments:]
            for i in tmp2:
                reply_is_author.append(i.split(":")[1])
            for i in range(len(user_link)-1):
                stockbar_names.append(stockbar_name)

        #发帖人相关            
        for suburl in user_link:
            html = decode(suburl)

            for item in html.body.find_all("div", class_ = "others_fans"):
                followers.append(item.find_all("span")[0].get_text())
                fans.append(item.find_all("span")[1].get_text())
            for item in html.body.find_all("div", class_ = "others_info"):
                total_visits.append(item.find_all("span")[0].get_text())
                today_visits.append(item.find_all("span")[1].get_text())
            if len(re.findall('\d+',str(html.body.find_all("div", class_="level2")[4])))==1:published_articles.append(0)
            else:published_articles.append(re.findall('\d+',str(html.body.find_all("div", class_="level2")[4]))[1])
            if len(re.findall('\d+',str(html.body.find_all('div', class_="level2")[5])))==1:published_comments.append(0)
            else:published_comments.append(re.findall('\d+',str(html.body.find_all('div', class_="level2")[5]))[1])


In [16]:
future_html = decode('http://guba.eastmoney.com/remenba.aspx?type=2')
for i in future_html.find_all('div', class_='allzhutilistb'):
    for item in i.find_all('a'):
        futures_code.append(item.attrs['href'][:-5])
print(futures_code)

links = []
check_time = []
stockbar_names = []
user_id = []
user_nickname = []
reviews = []
publish_time = []
reply_is_author = []
has_picture = []
post_click_count = []
post_comment_count = []
like_count = []
published_articles = []
published_comments = []
followers = []
fans = []
influence_power = []
user_age = []
total_visits = []
today_visits = []

columns = [check_time,stockbar_names,user_id,user_nickname,reviews,publish_time,reply_is_author,has_picture,
           post_click_count,post_comment_count,like_count,published_articles,published_comments,
           followers,fans,influence_power,user_age,total_visits,today_visits] 
col_name = ['查询时间','发帖吧','发帖id','发帖人','发帖内容','发帖时间','是否为主贴','是否有图片','阅读量','评论量','点赞量','发帖人发帖数','发帖人评论数','发帖人关注数','发帖人粉丝数','发帖人影响力','发帖人吧龄','发帖人总访问量','发帖人今日访问量']


In [31]:
baseurl = "http://guba.eastmoney.com/"
for code in futures_code:
    print(code)
    url = baseurl+code
    for page in range(1,101): #收集某个期货吧的前100页上所有帖子的链接
        soup = decode(url+'_'+str(page)+'.html')
        for item in soup.find_all('div',class_="articleh normal_post"):
            links.append(item.find('a').attrs['href'].replace('/',''))

links_url = []
for i in links:
    links_url.append("http://guba.eastmoney.com/"+i) 

f = open('links_url.txt','w')
f.write(str(links_url))

print(len(links_url)) #待爬链接数


list,qh
list,rb
list,wr
list,cu
list,al
list,ru
list,fu
list,zn
list,au
list,pb
list,baiyin
list,m
list,a
list,c
list,fshfesn
list,fshfeni
list,y
list,l
list,p
list,v
list,j
list,wt
list,ws
list,cf
list,sr
list,ta
list,ro
list,er
list,me
list,tc
list,bu
list,i
list,jd
list,jrx
list,bb
list,fb
list,pp
list,hc
list,lr
list,sf
list,sm
list,cs
list,gzqh
list,if01
list,if02
list,if03
list,if04
list,if05
list,if06
list,if07
list,if08
list,if09
list,if10
list,if11
list,if12
list,gzqh2
13314


In [ ]:
for url in links_url:
    html = decode(url)
    parse(html)

result = pd.DataFrame(columns).T
result.columns = col_name
result.to_excel('data_before_clear.xlsx')

In [5]:
##数据清洗
df = pd.read_excel('p1.xlsx')
#df.drop(axis = 1,labels='Unnamed: 0',inplace=True)
#删除特定发帖人
droped_rows = [i for i,x in enumerate(df['发帖人']) if df['发帖人'][i] in ['财经评论','东方财富网','期货小秘书']]
df.drop(droped_rows,axis=0)
#清洗评论内容
df['发帖内容'].fillna('空值',inplace=True)
null_list = df[(df['发帖内容']=='空值')].index.tolist()
df = df.drop(null_list)
reviews_cleared = []
for text in df['发帖内容']:
    reviews_cleared.append(text.replace(" ",'').replace("\n",'').replace("\r",''))
df['发帖内容'] = reviews_cleared
#数据格式变更
df['发帖吧'] = df['发帖吧'].apply(lambda x: str(x).replace('"',""))
df['发帖id'] = df['发帖id'].apply(str)
df['是否为主贴'] = df['是否为主贴'].apply(lambda x: 0 if x==False else 1)
df['发帖人吧龄'] = df['发帖人吧龄'].apply(lambda x: x.replace("年",""))
df['发帖人吧龄'] = df['发帖人吧龄'].apply(lambda x: str(float(x.replace("个月",""))/12) if "个月" in x else x)
df['发帖人吧龄'] = df['发帖人吧龄'].apply(lambda x: float(x.replace("天",""))/365 if "天" in x else float(x))
df = df.fillna('NAN').reset_index().drop(columns='index')
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4721 entries, 0 to 4720
Data columns (total 19 columns):
查询时间        4721 non-null object
发帖吧         4721 non-null object
发帖id        4721 non-null object
发帖人         4721 non-null object
发帖内容        4721 non-null object
发帖时间        4721 non-null object
是否为主贴       4721 non-null int64
是否有图片       4721 non-null float64
阅读量         4721 non-null object
评论量         4721 non-null object
点赞量         4721 non-null float64
发帖人发帖数      4721 non-null float64
发帖人评论数      4721 non-null float64
发帖人关注数      4721 non-null float64
发帖人粉丝数      4721 non-null float64
发帖人影响力      4721 non-null float64
发帖人吧龄       4721 non-null float64
发帖人总访问量     4721 non-null float64
发帖人今日访问量    4721 non-null float64
dtypes: float64(10), int64(1), object(8)
memory usage: 700.9+ KB
None


,查询时间,发帖吧,发帖id,发帖人,发帖内容,发帖时间,是否为主贴,是否有图片,阅读量,评论量,点赞量,发帖人发帖数,发帖人评论数,发帖人关注数,发帖人粉丝数,发帖人影响力,发帖人吧龄,发帖人总访问量,发帖人今日访问量
0,2019-12-08 09:24:38,棕榈油2006,2198024737861310.0,西门散户,这波棕榈怎么玩才能稳定赚钱？首先要明白怎么玩期货赚钱!我的思路一直是轻仓长线。做这波棕榈怎么...,2019-12-06 00:32:38,1,0.0,12502,158,87.0,599.0,1342.0,7.0,762.0,6.0,3.3,19891.0,578.0
1,2019-12-08 09:24:38,棕榈油2006,2198024737861310.0,西门散户,手机打字有错误理解,2019-12-06 00:36:25,1,0.0,NAN,NAN,0.0,599.0,1342.0,7.0,762.0,6.0,3.3,19891.0,578.0
2,2019-12-08 09:24:38,棕榈油2006,2606054690823332.0,只买垃圾168,高手、顶一个、镍吧里看到过你发言,2019-12-06 00:41:49,0,0.0,NAN,NAN,6.0,833.0,545.0,5.0,10.0,5.0,3.4,4083.0,22.0
3,2019-12-08 09:24:38,棕榈油2006,2198024737861310.0,西门散户,我认为做任何品种都是一样的的，只要有自己思考框架，交易框架，慢慢积少成多。,2019-12-06 00:43:24,1,0.0,NAN,NAN,1.0,599.0,1342.0,7.0,762.0,6.0,3.3,19891.0,578.0
4,2019-12-08 09:24:38,棕榈油2006,2198024737861310.0,西门散户,之前我做了一波镍的完全记录在沪镍吧，每周都有热帖，其实持仓过程比较煎熬的。还好沪镍一直走单边。,2019-12-06 00:45:16,1,0.0,NAN,NAN,1.0,599.0,1342.0,7.0,762.0,6.0,3.3,19891.0,578.0


In [6]:
# n日活跃度feature构建
time_delta = pd.to_datetime(df['查询时间'])-pd.to_datetime(df['发帖时间'])
time_delta = round(time_delta.apply(lambda x: float(re.findall('\d+', str(x))[0])+float(re.findall('\d+', str(x))[1])/24),4)
df['Time_delta'] = time_delta

n_num_reviews = np.zeros(len(df))

def n_day_activity(n):
    df_copy = df[df['Time_delta']<=n]
    _id = list(df_copy.groupby('发帖id', as_index=False).count()['发帖id'])
    num_reviews = list(df_copy.groupby('发帖id', as_index=False).count()['发帖人'])
    id_dict = dict(zip(_id,num_reviews))
    for i in range(len(df)):
        if df['发帖id'][i] not in id_dict: n_num_reviews[i] = 0
        else: n_num_reviews[i] = id_dict[df['发帖id'][i]]
    df['近n日发帖数'] = n_num_reviews
    return df

n = input()
n_day_activity(int(n)).to_excel('data_processed.xlsx')


3
